In [1]:
from src2.api import DataAPI , ModelPredictor , ModelHiddenExtractor
from pymysql.err import OperationalError
try:
    DataAPI.update()
except Exception:
    pass
DataAPI.prepare_predict_data()

ModelHiddenExtractor.update_hidden('gru_day' , model_submodels=['best'])
ModelPredictor.update_factors()

Update Files
Fri Sep 27 02:09:29 2024 : Updated ~ DB_information\calendar.feather Done! Cost 0.01 Secs
Fri Sep 27 02:09:29 2024 : Updated ~ DB_information\description.feather Done! Cost 0.05 Secs
Fri Sep 27 02:09:29 2024 : Updated ~ DB_information\st.feather Done! Cost 0.03 Secs
Fri Sep 27 02:09:33 2024 : Updated ~ DB_information\industry.feather Done! Cost 3.44 Secs
Fri Sep 27 02:09:39 2024 : Updated ~ DB_information\concepts.feather Done! Cost 6.52 Secs
Fri Sep 27 02:09:41 2024 : Updated ~ DB_models\longcl_exp\2024\longcl_exp.20240924.feather Done! Cost 0.21 Secs
Fri Sep 27 02:09:41 2024 : Updated ~ DB_models\risk_exp\2024\risk_exp.20240925.feather Done! Cost 0.16 Secs
Fri Sep 27 02:09:42 2024 : Updated ~ DB_models\risk_cov\2024\risk_cov.20240925.feather Done! Cost 0.11 Secs
Fri Sep 27 02:09:42 2024 : Updated ~ DB_models\risk_spec\2024\risk_spec.20240925.feather Done! Cost 0.06 Secs
Fri Sep 27 02:09:42 2024 : Updated ~ DB_models\risk_exp\2024\risk_exp.20240926.feather Done! Cost 0.17

100%|██████████| 316/316 [00:00<00:00, 1671.52it/s]


--------------------------------------------------------------------------------


 99%|█████████▊| 78/79 [00:03<00:00, 20.68it/s] 


OSError: [WinError 1265] 系统无法联系域控制器来为身份验证请求提供服务。请稍后再试。: '\\\\hfm-pubshare\\HFM各部门共享\\量化投资部\\龙昌伦\\Alpha\\gru_day_V0'

In [1]:
from src.api import DataAPI , Predictor

DataAPI.update()
DataAPI.prepare_predict_data()
Predictor.update_factors()

Update Files
Thu Sep 19 00:49:45 2024 : Updated ~ DB_information\calendar.feather Done! Cost 0.02 Secs
Thu Sep 19 00:49:45 2024 : Updated ~ DB_information\description.feather Done! Cost 0.03 Secs
Thu Sep 19 00:49:45 2024 : Updated ~ DB_information\st.feather Done! Cost 0.02 Secs
Thu Sep 19 00:49:48 2024 : Updated ~ DB_information\industry.feather Done! Cost 2.55 Secs
Thu Sep 19 00:49:53 2024 : Updated ~ DB_information\concepts.feather Done! Cost 5.54 Secs
Thu Sep 19 00:49:55 2024 : Updated ~ DB_models\longcl_exp\2024\longcl_exp.20240913.feather Done! Cost 0.10 Secs
Thu Sep 19 00:49:55 2024 : Updated ~ DB_models\risk_exp\2024\risk_exp.20240918.feather Done! Cost 0.05 Secs
Thu Sep 19 00:49:55 2024 : Updated ~ DB_models\risk_cov\2024\risk_cov.20240918.feather Done! Cost 0.02 Secs
Thu Sep 19 00:49:55 2024 : Updated ~ DB_models\risk_spec\2024\risk_spec.20240918.feather Done! Cost 0.01 Secs
Thu Sep 19 00:49:57 2024 : Updated ~ DB_trade\day\2024\day.20240918.feather Done! Cost 0.05 Secs
Thu S

Predictor(reg_model=RegModel(name=gru_avg,type=swabest,num=all,alias=gru_day_V1))

In [8]:
from src.data.tushare.download.info import Calendar , pro
renamer = {'cal_date' : 'calendar' ,'is_open'  : 'trade'}
fields = None
df = pro.trade_cal(exchange='SSE').rename(columns=renamer)
#df = df.sort_values('calendar').reset_index(drop = True)
df

""


In [10]:
# DataAPI.prepare_predict_data()
Predictor.update_factors()

Finish model [gru_day] predicting!
Finish model [gruRTN_day] predicting!
Finish model [gru_avg] predicting!
--------------------------------------------------------------------------------


In [2]:
from src.api import DataAPI , Predictor
# DataAPI.prepare_predict_data()
md = Predictor.update_factors()

Finish model [gru_day] predicting!
Finish model [gruRTN_day] predicting!
Finish model [gru_avg] predicting!
--------------------------------------------------------------------------------


In [ ]:
from src.api import ModelTestor
ModelTestor.new('ts_mixer').try_metrics()

Load  2 DataBlocks ...

... cost 0.22 secs
Align 2 DataBlocks ...... cost 0.21 secs
Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}
x shape is torch.Size([5064, 30, 6])
y shape is torch.Size([5064, 1])
Test Forward Success
metrics :  Metrics.MetricOutput(loss=tensor(1.0688, grad_fn=<AddBackward0>), score=-0.015420470386743546, loss_item=1.068833589553833, penalty=0.0, losses=tensor(1.0688, grad_fn=<ExpBackward0>))
Test Metrics Success
